In [7]:
pip install transformers tensorflow sentencepiece category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
from sklearn.metrics import mean_absolute_error, mean_squared_error
import copy
from numpy import unique
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
import category_encoders as ce
from keras.layers import Embedding
from keras.layers import concatenate
from keras.utils import plot_model
import math
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
from transformers import XLMRobertaTokenizer, TFAutoModel
from transformers import TFAutoModelForSequenceClassification
import matplotlib.pyplot as plt
#from keras.utils import plot_model
import os
from sklearn.preprocessing import RobustScaler
from huggingface_hub import from_pretrained_keras
from collections import Counter
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import losses
from transformers import pipeline
from keras.layers import Softmax
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers
from keras import backend as K
from sklearn.metrics import mean_absolute_error

In [9]:
#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
#load the data
df = pd.read_pickle("/content/drive/MyDrive/Thesis/1.data/4C.df_structured_unstructured").reset_index(drop = True)

In [12]:
def encode(df):
    #copy date column for sorting
    df["date copy"] = df["DT_AWARD"].copy(deep=True)

    #ENCODE COLUMNS
    base_n_encoder_cols = ["DT_AWARD", "TAL_LOCATION_NUTS", "LG_ORIG"]
    one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED", "ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", "B_ON_BEHALF", "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_EU_FUNDS", "CRIT_CODE", "CPV"]
    df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True, dtype=int)
    encoder = ce.BaseNEncoder(cols=base_n_encoder_cols, return_df=True, base=2)
    df = encoder.fit_transform(df)

    return df

In [13]:
def scale(df):
    numeric_cols = ["NUMBER_OFFERS", "AWARD_EST_VALUE_EURO", "CRIT_PRICE_WEIGHT"]
    scaler = RobustScaler()

    values = df[numeric_cols].values
    scaled_values = scaler.fit_transform(values)
    df.loc[:,numeric_cols] = np.array(scaled_values)

    return df

In [14]:
def encode_scale_split(df, train, validate, text_column):
    """encode scale and split dataset"""
    text_columns = ["short description", "title", "ac criterion", "short_descr_criterion", "combined text"]
    #ENCODE
    df = encode(df)
    df = df.sort_values(by = ["date copy"], axis = 0, ascending = True)
    train_indice = int(train * len(df))
    validate_indice = train_indice + int(validate * len(df))
    df = df.drop(columns = ["date copy"])

    train_set = df.iloc[:train_indice]
    val_set = df.iloc[train_indice:validate_indice]
    test_set = df.iloc[validate_indice:]

    #PERFORM SCALING
    train_set = scale(train_set)
    val_set = scale(val_set)
    test_set = scale(test_set)

    #SPLIT THE DATASET
    target_col = "AWARD_VALUE_EURO_FIN_1"
    drop_cols = [target_col] + text_columns
    X_train = train_set.drop(columns = drop_cols, axis=1).values
    X_train_text = train_set[text_column].values.flatten().tolist()
    y_train = train_set[target_col].values.astype(float)

    X_val = val_set.drop(columns = drop_cols, axis=1).values
    X_val_text = val_set[text_column].values.flatten().tolist()
    y_val = val_set[target_col].values.astype(float)

    X_test = test_set.drop(columns = drop_cols, axis=1).values
    X_test_text = test_set[text_column].values.flatten().tolist()
    y_test = test_set[target_col].values.astype(float)

    return X_train, X_train_text, y_train, X_val, X_val_text, y_val, X_test, X_test_text, y_test

In [15]:
def create_tokens(X_train_text, X_val_text, X_test_text, text_column):
  transformer_model_name = "jplu/tf-xlm-roberta-base"
  tokenizer = XLMRobertaTokenizer.from_pretrained(transformer_model_name)
  max_length = 128

  #Tokenize training set
  train_input_ids = tokenizer(X_train_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  train_attention_mask = tokenizer(X_train_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  #tokenize val set
  val_input_ids = tokenizer(X_val_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  val_attention_mask = tokenizer(X_val_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  #tokenize test set
  test_input_ids = tokenizer(X_test_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  test_attention_mask = tokenizer(X_test_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_train_input_ids", "wb") as f:
    pickle.dump(train_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_train_attention_mask", "wb") as f:
      pickle.dump(train_attention_mask, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_val_input_ids", "wb") as f:
      pickle.dump(val_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_val_attention_mask", "wb") as f:
      pickle.dump(val_attention_mask, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_test_input_ids", "wb") as f:
      pickle.dump(test_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_test_attention_mask", "wb") as f:
      pickle.dump(test_attention_mask, f)

In [16]:
#STATISTICS ON THE TEXT
def text_statistics(df, upper_bound, text_column):
  values = np.array([len(text.split(" ")) for text in df[text_column].values])
  df_text_stats = pd.DataFrame(values)
  value = np.sort(values)[int(upper_bound * len(values))-1: int(upper_bound * len(values))]

  return df_text_stats.describe(), value

In [17]:
def load_tokenized_text(text_col):
  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_train_input_ids", "rb") as f:
    train_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_train_attention_mask", "rb") as f:
    train_attention_mask = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_val_input_ids", "rb") as f:
    val_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_val_attention_mask", "rb") as f:
    val_attention_mask = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_test_input_ids", "rb") as f:
    test_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_test_attention_mask", "rb") as f:
    test_attention_mask = pickle.load(f)

  return train_input_ids, train_attention_mask, val_input_ids, val_attention_mask, test_input_ids, test_attention_mask

In [18]:
def RAE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum(abs(true_value - pred_value) for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true) for true_value in y_true)
    rae = float((delta_pred / delta_mean).item())
    return rae

In [19]:
def RMSE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum((true_value - pred_value)**2 for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true)**2 for true_value in y_true)
    rmse = float((delta_pred / delta_mean).item())
    return rmse

In [20]:
#LOAD TOKENIZED DATA
X_train, X_train_text, y_train, X_val, X_val_text, y_val, X_test, X_test_text, y_test = encode_scale_split(df, 0.6, 0.2, text_column = "combined text");
#create_tokens(X_train_text, X_val_text, X_test_text, text_column = "test_sequence_length_128")
train_input_ids, train_attention_mask, val_input_ids, val_attention_mask, test_input_ids, test_attention_mask = load_tokenized_text("combined text")

<ipython-input-13-98fe2c74d091>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-13-98fe2c74d091>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-13-98fe2c74d091>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [ ]:
#DEFINE VARIABLES
input_dim_num_cat=X_train.shape[1]
max_length = 128

import math
#define path for saving model
checkpoint_path = "/content/drive/MyDrive/Thesis/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#specify model parameters
batch_size = 32
num_epochs = 5

n_batches = len(X_train) / batch_size
n_batches = math.ceil(n_batches)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1, save_freq=n_batches)

#MAKE THE FINAL MODEL
input_num_cat = Input(shape=input_dim_num_cat)
x = Dense(128, activation = "relu")(input_num_cat)
x = Dense(32, activation = "relu")(x)
x = Dense(4, activation = "relu")(x)
num_cat_output = Dense(1, activation = "relu")(x)

#INITIALIZE THE TRANSFORMER MODEL
transformer_model_name = "jplu/tf-xlm-roberta-base"
transformer = TFAutoModelForSequenceClassification.from_pretrained(transformer_model_name, num_labels=1)
transformer.layers[0].trainable = False

#DEFINE INPUTS AND FEED TO THE TRANSFORMER MODEL
input_ids = tf.keras.Input(shape=(max_length,), dtype='int32')
attention_mask = tf.keras.Input(shape=(max_length,), dtype='int32')
transformer_output = transformer(input_ids, attention_mask).logits

#COMBINE THE MODEL OUTPUTS
concat = concatenate([num_cat_output, transformer_output])
final_layer = Dense(1, activation = "linear")(concat)

#create final model
final_model = Model(inputs=[{"input_ids": input_ids, "attention_mask": attention_mask}, input_num_cat],
                    outputs=[final_layer],
                    name="merged")

#DEFINE LEARNING RATE DECAY FOR OPTIMIZATION
num_train_steps = len(y_train) / batch_size * num_epochs
lr_scheduler = PolynomialDecay(initial_learning_rate=0.0001,
                               end_learning_rate=0.000001,
                               decay_steps=num_train_steps)

#define custom loss function for combining classification with regression
loss_function = keras.losses.MeanSquaredError(reduction="sum_over_batch_size", name="mean_squared_error")
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
metrics = ["mse", "mae", "R2Score"]

final_model.compile(loss=loss_function,
                   optimizer=optimizer,
                    metrics=metrics)

history_final_model = final_model.fit(x=[[train_input_ids, train_attention_mask], X_train], y=y_train,
                validation_data=([[val_input_ids, val_attention_mask], X_val], y_val),
                epochs=num_epochs,
                batch_size = batch_size,
                callbacks=[cp_callback])

final_model.save_weights('/content/drive/MyDrive/Thesis/combined_model_weights.keras')

In [22]:
y_pred = final_model.predict([[test_input_ids, test_attention_mask], X_test])
mae_test = mean_absolute_error(y_test, y_pred)
mse_test = mean_squared_error(y_test, y_pred)
rae_test = RAE(y_test, y_pred)
rmse_test = RMSE(y_test, y_pred)

r2_metric = tf.keras.metrics.R2Score()
r2_metric.update_state(y_test.reshape(-1, 1), y_pred)
r2_result = r2_metric.result()
r2_result = r2_result.numpy()

results_total = {}
results_total["language_model"] = {"history": history_final_model,
                                    "mae_test": mae_test,
                                    "mse_test": mse_test,
                                    "r2_results": r2_result,
                                    "rae_test": rae_test,
                                    "rmse_test": rmse_test}

244/244 [==============================] - 44s 179ms/step


In [23]:
results_total

{'language_model': {'history': <keras.src.callbacks.History at 0x7c4f3599dd50>,
  'mae_test': 628244.6667203327,
  'mse_test': 1215207770315.3708,
  'r2_results': -0.48085046,
  'rae_test': 1.0828227996826172,
  'rmse_test': 1.4808497428894043}}

In [ ]:
#DEFINE VARIABLES
input_dim_num_cat=X_train.shape[1]
max_length = 256

import math
#define path for saving model
checkpoint_path = "/content/drive/MyDrive/Thesis/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#specify model parameters
batch_size = 32
num_epochs = 20

n_batches = len(X_train) / batch_size
n_batches = math.ceil(n_batches)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1, save_freq=n_batches)

#MAKE THE FINAL MODEL
input_num_cat = Input(shape=input_dim_num_cat)
x = Dense(32, activation = "relu")(input_num_cat)
x = Dropout(rate=0.2)(x)
x = Dense(8, activation = "relu")(x)
x = Dropout(rate=0.2)(x)
x = Dense(32, activation = "relu")(x)
x = Dropout(rate=0.2)(x)
num_cat_output = Dense(1, activation = "relu")(x)

#INITIALIZE THE TRANSFORMER MODEL
transformer_model_name = "jplu/tf-xlm-roberta-base"
transformer = TFAutoModelForSequenceClassification.from_pretrained(transformer_model_name, num_labels=1)
transformer.layers[0].trainable = False

#DEFINE INPUTS AND FEED TO THE TRANSFORMER MODEL
input_ids = tf.keras.Input(shape=(max_length,), dtype='int32')
attention_mask = tf.keras.Input(shape=(max_length,), dtype='int32')
transformer_output = transformer(input_ids, attention_mask).logits

#COMBINE THE MODEL OUTPUTS
concatentate = concatenate([num_cat_output, transformer_output])
final_layer = Dense(1, activation = "linear")(concatentate)

#create final model
final_model = Model(inputs=[{"input_ids": input_ids, "attention_mask": attention_mask}, input_num_cat],
                    outputs=[final_layer],
                    name="merged")

#DEFINE LEARNING RATE DECAY FOR OPTIMIZATION
num_train_steps = len(y_train) / batch_size * num_epochs
lr_scheduler = PolynomialDecay(initial_learning_rate=0.0001,
                               end_learning_rate=0.000001,
                               decay_steps=num_train_steps)

#define custom loss function for combining classification with regression
loss_function = keras.losses.MeanSquaredError(reduction="sum_over_batch_size", name="mean_squared_error")
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
metrics = ["mse", "mae", "R2Score"]

final_model.compile(loss=loss_function,
                   optimizer=optimizer,
                    metrics=metrics)

history_final_model = final_model.fit(x=[[train_input_ids, train_attention_mask], X_train], y=y_train,
                validation_data=([[val_input_ids, val_attention_mask], X_val], y_val),
                epochs=num_epochs,
                batch_size = batch_size,
                callbacks=[cp_callback])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
731/732 [============================>.] - ETA: 8s - loss: 1380824121344.0000 - mse: 1380824121344.0000 - mae: 659028.5000 - r2_score: -0.4589 
Epoch 1: saving model to /content/drive/MyDrive/Thesis/checkpoint.ckpt
732/732 [==============================] - 7879s 11s/step - loss: 1380408360960.0000 - mse: 1380408360960.0000 - mae: 658974.1875 - r2_score: -0.4590 - val_loss: 1255519158272.0000 - val_mse: 1255519158272.0000 - val_mae: 625319.1875 - val_r2_score: -0.4523
Epoch 2/20
355/732 [=============>................] - ETA: 51:30 - loss: 1437100933120.0000 - mse: 1437100933120.0000 - mae: 669280.2500 - r2_score: -0.4528